In [1]:
import requests
import csv
from ltr.data import CorpusApi, Config
import numpy as np
import json

In [2]:
queries = CorpusApi.getValidationQueriesAsDict()

In [3]:
evaluationMap = {}

with open('data/validation/docv2_trec2020_qrels.txt', 'r') as txt:
    txtReader = csv.reader(txt, delimiter=' ')
    for line in txtReader:
        queryMap = evaluationMap.get(line[0], {})
        forwardIndexList = queryMap.get(line[3],[])
        forwardIndexList.append(line[2])
        queryMap[line[3]] = forwardIndexList
        reverseIndexMap = queryMap.get('reverse', {})
        reverseIndexMap[line[2]] = line[3]
        queryMap['reverse'] = reverseIndexMap
        
        evaluationMap[line[0]] = queryMap
        

In [4]:
from typing import List


def reciprocalRank(queryId: int, evalList: List) -> float:
    
    rank = 1
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    for docId in evalList:
        v = lookupMap.get(docId)
        if v is None or int(v) == 0:
            rank += 1
        else:
            break

    return 1/rank if rank <= 100 else 0
    

In [5]:
def averagePrecision(queryId: int, evalList: List) -> float:
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    foundDocs = 0
    
    sum = 0
    
    for i, docId in enumerate(evalList):
        v = lookupMap.get(docId)
        if v is not None and int(v) != 0:
            foundDocs += 1
            sum += (foundDocs / (i + 1))
            
    
    return sum / len(evalList)

In [6]:
import math

def normalizedDiscountedCumulativeGain(queryId: int, evalList: List) -> float:
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    # create gain list
    gainList = [int(lookupMap.get(docId, 0)) for docId in evalList]
    #print(gainList)
    
    # create optimal gain list
    keys = list(evaluationMap[queryId].keys()).copy()
    
    if 'reverse' in keys:
        keys.remove('reverse')
    
    if 'rr' in keys:
        keys.remove('rr')
    
    if 'ap' in keys:
        keys.remove('ap')
    
    if 'ndcg' in keys:
        keys.remove('ndcg')
    
    if 'ncg' in keys:
        keys.remove('ncg')
    
    keys = [int(idx) for idx in keys]
    
    optimalGainList = []
    for key in sorted(keys, reverse=True):
        gains = [key for value in range(0,len(evaluationMap[queryId][str(key)]))]
        optimalGainList.extend(gains)
    
    optimalGainList = optimalGainList[:len(evalList)]
    
    dcg = 0
    
    for i in range(0, len(gainList)):
        dcg += gainList[i] / math.log2(i + 2)
        
    idcg = 0
    for i in range(0, len(optimalGainList)):
        idcg += optimalGainList[i] / math.log2(i + 2)
        
    return dcg / idcg
    

In [7]:


def normalizedCumulativeGain(queryId: int, evalList: List) -> float:
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    # create gain list
    gainList = [int(lookupMap.get(docId, 0)) for docId in evalList]
    #print(gainList)
    
    # create optimal gain list
    keys = list(evaluationMap[queryId].keys()).copy()
    
    if 'reverse' in keys:
        keys.remove('reverse')
    
    if 'rr' in keys:
        keys.remove('rr')
    
    if 'ap' in keys:
        keys.remove('ap')
    
    if 'ndcg' in keys:
        keys.remove('ndcg')
    
    if 'ncg' in keys:
        keys.remove('ncg')
    
    keys = [int(idx) for idx in keys]
    
    optimalGainList = []
    for key in sorted(keys, reverse=True):
        gains = [key for value in range(0,len(evaluationMap[queryId][str(key)]))]
        optimalGainList.extend(gains)
    
    optimalGainList = optimalGainList[:len(evalList)]
    

    return sum(gainList) / sum(optimalGainList)

In [9]:
from nltk.tokenize import word_tokenize

for i, key in enumerate(list(evaluationMap.keys())[:]):
    try:
        # get query
        query = queries[key]
        
        
        # prepare for ltr
        queryTokens = word_tokenize(query)
        tcqtValues = ', '.join([f"if(termfreq(title,'{t}'),1,0)" for t in queryTokens])
        hcqtValues = ', '.join([f"if(termfreq(headings,'{t}'),1,0)" for t in queryTokens])
        bcqtValues = ', '.join([f"if(termfreq(body,'{t}'),1,0)" for t in queryTokens])
        dcqtValues = ', '.join([f"if(termfreq(_text_,'{t}'),1,0)" for t in queryTokens])
        ttfValues = ', '.join([f"tf(title,'{t}')" for t in queryTokens])
        htfValues = ', '.join([f"tf(headings,'{t}')" for t in queryTokens])
        btfValues = ', '.join([f"tf(body,'{t}')" for t in queryTokens])
        dtfValues = ', '.join([f"tf(_text_,'{t}')" for t in queryTokens])
        tidfValues = ', '.join([f"idf(title,'{t}')" for t in queryTokens])
        hidfValues = ', '.join([f"idf(headings,'{t}')" for t in queryTokens])
        bidfValues = ', '.join([f"idf(body,'{t}')" for t in queryTokens])
        didfValues = ', '.join([f"idf(_text_,'{t}')" for t in queryTokens])
        ttfidfValues = ', '.join([f"product(tf(title,'{t}'),idf(title,'{t}'))" for t in queryTokens])
        htfidfValues = ', '.join([f"product(tf(headings,'{t}'),idf(headings,'{t}'))" for t in queryTokens])
        btfidfValues = ', '.join([f"product(tf(body,'{t}'),idf(body,'{t}'))" for t in queryTokens])
        dtfidfValues = ', '.join([f"product(tf(_text_,'{t}'),idf(_text_,'{t}'))" for t in queryTokens])
        
        
        
        
        # get result from solr
        request = {
            "fields": "id",
            "limit": 100,
            "params": {
                "rq": "{!ltr reRankDocs=500 " + 
                    f"""
                  efi.keywords=\"{query}\" 
                  efi.tcqt_values=\"  {tcqtValues}
                  \" 
                  efi.hcqt_values=\"  {hcqtValues}
                  \"
                  efi.bcqt_values=\"  {bcqtValues}
                  \"
                  efi.dcqt_values=\"  {dcqtValues}
                  \"
                  efi.query_terms_length={len(queryTokens)}
                  efi.ttf_values=\"  {ttfValues}
                  \" 
                  efi.htf_values=\"  {htfValues}
                  \" 
                  efi.btf_values=\"  {btfValues}
                  \" 
                  efi.dtf_values=\"  {dtfValues}
                  \" 
                  efi.tidf_values=\"  {tidfValues}
                  \" 
                  efi.hidf_values=\"  {hidfValues}
                  \" 
                  efi.bidf_values=\"  {bidfValues}
                  \" 
                  efi.didf_values=\"  {didfValues}
                  \" 
                  efi.ttfidf_values=\"  {ttfidfValues}
                  \" 
                  efi.htfidf_values=\"  {htfidfValues}
                  \" 
                  efi.btfidf_values=\"  {btfidfValues}
                  \" 
                  efi.dtfidf_values=\"  {dtfidfValues}
                  \" 
          """
                +"model=thesis-ranknet }",
                "qf": "title headings body",
                "defType": "dismax",
                "q": query
            }
        }
        
        if key == ' 10000':
            print(json.dumps(request, indent=2))
        
        response = requests.post(f'http://localhost:8983/solr/thesis-ltr/select', json=request)
        
        queryResult = [doc['id'] for doc in response.json()["response"]["docs"]]
        
        
        # calculate metrics
        # Reciprocal Rank
        rr = reciprocalRank(key, queryResult)
        evaluationMap[key]['rr'] = rr
        print(f'RR: {rr}')
        # Normalized Discounted Cumulative Gains (NDCG)
        ndcg = normalizedDiscountedCumulativeGain(key, queryResult[:10])
        evaluationMap[key]['ndcg'] = ndcg
        print(f'NDCG: {ndcg}')
        # Normalized Cumulative Gains (NCG)
        ncg = normalizedCumulativeGain(key, queryResult)
        evaluationMap[key]['ncg'] = ncg
        print(f'NCG: {ncg}')
        # Average Precision (AP)
        ap = averagePrecision(key, queryResult)
        evaluationMap[key]['ap'] = ap
        print(f'AP: {ap}')
        
        print(f'processed {i}/{len(list(evaluationMap.keys())[:])} {key} query: {query}')
    except KeyError as err:
        # ignore the queries that are not in the validation queries list
        pass

RR: 0
NDCG: 0.0
NCG: 0.0
AP: 0.0
processed 0/45 42255 query: average salary for dental hygienist in nebraska
RR: 0.16666666666666666
NDCG: 0.07839826897867533
NCG: 0.345679012345679
AP: 0.024693591288073753
processed 1/45 47210 query: average wedding dress alteration cost
RR: 1.0
NDCG: 0.08069862424496003
NCG: 0.12162162162162163
AP: 0.014819723049729092
processed 2/45 67316 query: can fever cause miscarriage early pregnancy
RR: 0.3333333333333333
NDCG: 0.3086147415907597
NCG: 0.07894736842105263
AP: 0.014083333333333331
processed 3/45 135802 query: definition of laudable
RR: 0.1
NDCG: 0.037665676876396506
NCG: 0.25
AP: 0.0031543016370602583
processed 4/45 156498 query: do google docs auto save
RR: 0.3333333333333333
NDCG: 0.09650766274495248
NCG: 0.24299065420560748
AP: 0.027264958362455235
processed 5/45 169208 query: does mississippi have an income tax
RR: 1.0
NDCG: 0.22009176629808014
NCG: 0.08
AP: 0.011332127787823991
processed 6/45 174463 query: dog day afternoon meaning
RR: 1.0


In [ ]:
bm25_rr = 0
bm25_ap = 0
bm25_ndcg = 0
bm25_ncg = 0
counter = 0
for key in list(evaluationMap.keys()):
    try:
        bm25_rr += evaluationMap[key]['rr']
        bm25_ap += evaluationMap[key]['ap']
        bm25_ndcg += evaluationMap[key]['ndcg']
        bm25_ncg += evaluationMap[key]['ncg']
        counter += 1
    except KeyError:
        pass

In [ ]:
bm25_rr = bm25_rr/counter
bm25_ap = bm25_ap/counter
bm25_ndcg = bm25_ndcg/counter
bm25_ncg = bm25_ncg/counter

print(f'exp_bm25: rr:{bm25_rr} / ap:{bm25_ap} / ndcg@10:{bm25_ndcg} / ncg:{bm25_ncg}')
